# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [48]:
# imports
import os
import requests
import json
import ollama
import time
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [31]:
# constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [32]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [121]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
"""
line_of_code=input("Please enter line of code that needs explanation:")
question += line_of_code
print (question)

Please enter line of code that needs explanation: yield from {book.get("author") from book in books if book.get("author")}



Please explain what this code does and why:
yield from {book.get("author") from book in books if book.get("author")}


In [122]:
system_prompt = "You are a technical assistant who can review a line of code and explain what this line of code does, \
with a technical breakdown of each piece of the code, and a one line summary in plain english at the end."

In [119]:
# Get gpt-4o-mini to answer, with streaming
def tech_copilot_gpt(question,model):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    

In [123]:
tech_copilot_gpt(question,MODEL_GPT)

Let's break down the line of code provided:

1. **`yield from`**: This is a Python keyword used in generator functions. It allows a generator to yield all values from another iterable. In this case, it indicates that the function is yielding values from an iterable that follows.

2. **`{book.get("author") for book in books if book.get("author")}`**: This part is a set comprehension. Set comprehensions are used to construct a set based on a given iterable. They are similar to list comprehensions but create a set instead of a list. 

    a. **`book.get("author")`**: This is calling the `get` method on each `book` dictionary within the iterable `books`. It retrieves the value associated with the key `"author"`. If the key does not exist, `get` would return `None`, which is handled by the next part.
    
    b. **`for book in books`**: This iterates over each `book` in a collection named `books`. `books` is expected to be an iterable (like a list or set) of dictionaries, where each dictionary represents a book.

    c. **`if book.get("author")`**: This conditional filters out books where the `get` method does not return a truthy value (i.e., it filters out cases where the author is `None` or does not exist). Thus, only books with valid authors are included.

3. **Summary**: This code efficiently collects unique authors from a collection of book dictionaries, yielding them one by one.

### One-line summary:
This code yields unique authors from a collection of books, filtering out any entries without an author.

In [125]:
# Get Llama 3.2 to answer

def tech_copilot_llama_1(question,model):
    payload = {
            "model": model,
            "messages":[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question},
            ],
            "stream": True 
        }
    
    stream = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream.iter_lines():
        decoded_data = chunk.decode("utf-8")
        parsed_json = json.loads(decoded_data)  # Parse JSON
        response += parsed_json["message"]["content"] or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

def tech_copilot_llama_2(question,model):
    messages =[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question},
            ]
    response = ollama.chat(model=model, messages=messages)
    reply = response['message']['content']
    display(Markdown(reply))

In [126]:
tech_copilot_llama_2(question,MODEL_LLAMA)

Here's a technical breakdown of the code:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line of code is using a feature called "Generator Expression" in Python, which allows you to create and iterate over an iterable (such as a list or dictionary) without having to explicitly define a function.

**Here's what each part does:**

* `{...}` creates a new iterator that yields values from the expression inside.
* `book.get("author")` is used to extract the value of the "author" key from each book dictionary. If the key doesn't exist, it returns `None`.
* `for book in books` iterates over the list of books (`books`).
* `if book.get("author")` filters out books that don't have an author.
* `yield from {...}` is a syntax sugar for a loop that yields values from another iterator. In this case, it's yielding all values from the generator expression inside `{...}`.

So, when you put it all together, this line of code:

1. Filters out books without authors.
2. Yields the author name for each remaining book.

**Why is this useful?**

This code is likely being used in a context where we want to process books with authors, but don't care about non-author books. By using a generator expression and `yield from`, we can efficiently iterate over only the relevant data without having to store it all in memory at once.

**One line summary:** This code generates a sequence of author names from a list of books, filtering out books without authors.